##  Cross-Encoder + LightGBM 앙상블 모델 (데이터 누수 해결 버전)

이 노트북은 K-겹 교차 검증을 적용하여 데이터 누수 문제를 해결한 앙상블 모델을 구축합니다.

### **✅ 수정된 방법론**
1.  **특징 엔지니어링**: 기존과 동일하게 수치 및 범주형 특징을 생성합니다.
2.  **Cross-Encoder 교차 검증 예측 (Out-of-Fold Prediction)**: 5-겹 교차 검증을 사용하여 데이터 누수가 없는 'semantic score'(`ce_score`)를 생성합니다. 각 데이터 포인트의 점수는 해당 데이터가 포함되지 않은 훈련 데이터로 학습된 모델에 의해 예측됩니다.
3.  **LightGBM 훈련**: 교차 검증으로 생성된 'semantic score'와 다른 특징들을 결합하여 최종 LGBM 모델을 훈련합니다.
4.  **최종 모델 저장**: 추론에 사용할 최종 Cross-Encoder(전체 데이터로 학습)와 LGBM 모델, 그리고 전처리기들을 저장합니다.

In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import joblib
import re
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from sentence_transformers import CrossEncoder, InputExample
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from scipy.sparse import hstack
import lightgbm as lgb

# GPU 사용 가능 여부 확인
print(f"GPU 사용 가능: {torch.cuda.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

In [ ]:
# ==========================================================
# 📊 데이터 로드 및 기본 정보 확인
# ==========================================================
train_df = pd.read_csv('train.csv')

print(f"🔍 데이터 형태: {train_df.shape}")
print(f"🎯 타겟 분포: {train_df['rule_violation'].value_counts().to_dict()}")
print("\n📋 데이터 샘플:")
display(train_df.head())

In [ ]:
# ==========================================================
# 🛠️ 특징 엔지니어링 및 입력 준비 함수 정의 (기존과 동일)
# ==========================================================
def count_urls(text):
    return len(re.findall(r'https?://\S+|www\.\S+', str(text)))

def count_exclaims(text):
    return str(text).count('!')

def count_questions(text):
    return str(text).count('?')

def upper_ratio(text):
    s = str(text)
    letters = [c for c in s if c.isalpha()]
    if not letters:
        return 0.0
    upp = sum(1 for c in letters if c.isupper())
    return upp / len(letters)

def repeat_char_max(text):
    longest = 1
    last = ''
    cur = 0
    for ch in str(text):
        if ch == last:
            cur += 1
        else:
            longest = max(longest, cur)
            cur = 1
            last = ch
    longest = max(longest, cur)
    return longest

def jaccard_similarity(text1, text2):
    set1 = set(str(text1).lower().split())
    set2 = set(str(text2).lower().split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0.0

def create_features(df):
    df = df.copy()
    df['body_len'] = df['body'].astype(str).str.len()
    df['rule_len'] = df['rule'].astype(str).str.len()
    df['body_words'] = df['body'].astype(str).str.split().str.len()
    df['url_cnt'] = df['body'].apply(count_urls)
    df['exc_cnt'] = df['body'].apply(count_exclaims)
    df['q_cnt'] = df['body'].apply(count_questions)
    df['upper_rt'] = df['body'].apply(upper_ratio)
    df['rep_run'] = df['body'].apply(repeat_char_max)
    df['rule_body_jaccard'] = [jaccard_similarity(rule, body) for rule, body in zip(df['rule'], df['body'])]
    return df

def prepare_cross_encoder_input(rule, body, positive_ex1=None, negative_ex1=None):
    rule_text = str(rule).strip()
    comment_text = str(body).strip()
    examples_text = ""
    if pd.notna(positive_ex1) and str(positive_ex1).strip():
        examples_text += f" [긍정예시] {str(positive_ex1).strip()}"
    if pd.notna(negative_ex1) and str(negative_ex1).strip():
        examples_text += f" [부정예시] {str(negative_ex1).strip()}"
    full_input = f"[규칙] {rule_text}{examples_text} [댓글] {comment_text}"
    return full_input

# 특징 생성 실행
print("🔧 기본 특징 생성 중...")
train_df_featured = create_features(train_df)
print("✅ 기본 특징 생성 완료!")

# Cross-Encoder 입력 및 레이블 준비
ce_inputs = []
for idx, row in tqdm(train_df.iterrows(), total=len(train_df), desc="CE 입력 데이터 준비"):
    ce_input = prepare_cross_encoder_input(
        row['rule'], row['body'], row.get('positive_example_1'), row.get('negative_example_1')
    )
    ce_inputs.append(ce_input)

ce_inputs = np.array(ce_inputs)
labels = train_df['rule_violation'].values
print(f"✅ {len(ce_inputs)}개의 Cross-Encoder 입력 및 레이블 준비 완료")

### ## 🌟 1단계: Cross-Encoder로 누수 없는 특징(ce_score) 생성 (K-Fold)

In [ ]:
from sentence_transformers.evaluation import CESoftmaxAccuracyEvaluator
from sklearn.model_selection import train_test_split

# =========================================================
# 🚀 1단계: Cross-Encoder 최적화 설정
# =========================================================
N_SPLITS = 5
# 1. 모델명 변경
MODEL_NAME = 'microsoft/deberta-v3-base' 

# 2. 하이퍼파라미터 설정
LEARNING_RATE = 2e-5
BATCH_SIZE = 16 # VRAM 충분하면 32로 변경
EPOCHS = 5      # Evaluator가 최적점을 찾으므로 넉넉하게 설정
WEIGHT_DECAY = 0.01
# =========================================================

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
oof_ce_scores = np.zeros(len(train_df))

print(f"🚀 {N_SPLITS}-겹 교차 검증 (모델: {MODEL_NAME})")

for fold, (train_idx, val_idx) in enumerate(skf.split(ce_inputs, labels)):
    print(f'\n===== Fold {fold+1}/{N_SPLITS} =====')
    
    X_train_fold, X_val_fold = ce_inputs[train_idx], ce_inputs[val_idx]
    y_train_fold, y_val_fold = labels[train_idx], labels[val_idx]
    
    # 각 Fold의 훈련 데이터 중 10%를 모델 성능 측정을 위한 검증셋으로 분리
    train_texts, eval_texts, train_labels, eval_labels = train_test_split(
        X_train_fold, y_train_fold, test_size=0.1, random_state=42, stratify=y_train_fold
    )
    
    # 훈련용 InputExample 생성
    train_examples = []
    for i in range(len(train_texts)):
        text, label = train_texts[i], train_labels[i]
        rule_part, comment_part = text.split('[댓글]', 1) if '[댓글]' in text else (text, "")
        train_examples.append(InputExample(texts=[rule_part.strip(), comment_part.strip()], label=float(label)))

    # 검증용 InputExample 생성
    eval_examples = []
    for i in range(len(eval_texts)):
        text, label = eval_texts[i], eval_labels[i]
        rule_part, comment_part = text.split('[댓글]', 1) if '[댓글]' in text else (text, "")
        eval_examples.append(InputExample(texts=[rule_part.strip(), comment_part.strip()], label=float(label)))

    # 모델 초기화
    ce_model_fold = CrossEncoder(MODEL_NAME, num_labels=1, device=device)
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=BATCH_SIZE)
    warmup_steps = int(len(train_dataloader) * 0.1)

    # 💡 성능 평가기(Evaluator) 설정
    # 훈련 중 주기적으로 검증셋의 정확도를 평가하여 최고 성능 모델을 저장합니다.
    evaluator = CESoftmaxAccuracyEvaluator.from_input_examples(eval_examples, name=f'fold-{fold}-eval')
    
    print(f"Fold {fold+1} 훈련 시작...")
    ce_model_fold.fit(
        train_dataloader=train_dataloader,
        epochs=EPOCHS,
        warmup_steps=warmup_steps,
        evaluator=evaluator,
        evaluation_steps=int(len(train_dataloader) * 0.2), # 1 에폭 당 5번 평가
        optimizer_params={'lr': LEARNING_RATE}, # 3. Learning Rate 설정
        weight_decay=WEIGHT_DECAY,             # 4. Weight Decay 설정
        output_path=f'./model_output/best_model_fold_{fold}', # 최고 성능 모델 저장
        show_progress_bar=True
    )
    
    # OOF 예측 시에는 가장 성능이 좋았던 모델을 다시 불러와서 사용
    best_model = CrossEncoder(f'./model_output/best_model_fold_{fold}')
    
    print(f"Fold {fold+1} 예측 중...")
    val_predict_inputs = []
    for text in X_val_fold:
        rule_part, comment_part = text.split('[댓글]', 1) if '[댓글]' in text else (text, "")
        val_predict_inputs.append([rule_part.strip(), comment_part.strip()])
        
    predictions = best_model.predict(val_predict_inputs, show_progress_bar=True)
    oof_ce_scores[val_idx] = predictions

print("\n✅ 모든 폴드에 대한 Out-of-Fold 예측 완료!")
train_df_featured['ce_score'] = oof_ce_scores

### ## 🌳 2단계: LightGBM 모델 훈련

In [ ]:
print("🔧 LGBM 모델을 위한 특징 스케일링 및 인코딩 중...")

# 1. 수치 특징 (OOF 예측 점수인 ce_score 포함)
numerical_cols = [
    'body_len', 'rule_len', 'body_words', 'url_cnt', 'exc_cnt', 'q_cnt',
    'upper_rt', 'rep_run', 'rule_body_jaccard', 
    'ce_score' 
]
scaler = StandardScaler()
numerical_features = scaler.fit_transform(train_df_featured[numerical_cols])

# 2. 범주형 특징
categorical_cols = ['subreddit']
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
categorical_features = onehot_encoder.fit_transform(train_df_featured[categorical_cols])

# 3. 모든 특징 결합
X_lgbm = hstack([numerical_features, categorical_features])
y_lgbm = train_df_featured['rule_violation'].values

print(f"✅ LGBM 훈련 데이터 준비 완료. 최종 형태: {X_lgbm.shape}")

# 4. LGBM 모델 훈련
print("\n🚀 LightGBM 모델 훈련 시작...")
lgbm_model = lgb.LGBMClassifier(
    objective='binary', metric='auc', n_estimators=1000, learning_rate=0.05,
    num_leaves=31, max_depth=-1, random_state=42, n_jobs=-1,
    colsample_bytree=0.8, subsample=0.8
)

lgbm_model.fit(X_lgbm, y_lgbm, 
             eval_set=[(X_lgbm, y_lgbm)],
             eval_metric='auc',
             callbacks=[lgb.early_stopping(100, verbose=False)])

print("✅ LightGBM 모델 훈련 완료!")

### ## 💾 3단계: 최종 모델 및 전처리기 저장

In [ ]:
# 추론 시 test 데이터에 적용하기 위해 Cross-Encoder를 전체 데이터로 다시 학습
print("🚀 추론용 최종 Cross-Encoder 모델을 전체 데이터로 학습합니다...")

final_cross_encoder = CrossEncoder(MODEL_NAME, num_labels=1, device=device)

# 전체 훈련 데이터 준비
final_train_examples = []
for i in range(len(ce_inputs)):
    text = ce_inputs[i]
    rule_part, comment_part = text.split('[댓글]', 1) if '[댓글]' in text else (text, "")
    final_train_examples.append(InputExample(texts=[rule_part.strip(), comment_part.strip()], label=float(labels[i])))

final_train_dataloader = DataLoader(final_train_examples, shuffle=True, batch_size=16)
warmup_steps = int(len(final_train_dataloader) * 0.1)

final_cross_encoder.fit(
    train_dataloader=final_train_dataloader,
    epochs=1, # 전체 데이터이므로 1 에폭으로 충분
    warmup_steps=warmup_steps,
    show_progress_bar=True
)
print("✅ 최종 Cross-Encoder 모델 학습 완료!")

# ==========================================================
# 💾 최종 모델 및 전처리 객체 저장
# ==========================================================
output_dir = './model_output'
os.makedirs(output_dir, exist_ok=True)

# 1. Cross-Encoder 모델 저장
ce_output_path = os.path.join(output_dir, 'final_cross_encoder_model')
final_cross_encoder.save(ce_output_path)
print(f"✅ Cross-Encoder 모델 저장 완료: {ce_output_path}")

# 2. LGBM 모델 저장
joblib.dump(lgbm_model, os.path.join(output_dir, 'lgbm_model.pkl'))
print(f"✅ LGBM 모델 저장 완료: {os.path.join(output_dir, 'lgbm_model.pkl')}")

# 3. Scaler 저장
joblib.dump(scaler, os.path.join(output_dir, 'scaler.pkl'))
print(f"✅ Scaler 저장 완료: {os.path.join(output_dir, 'scaler.pkl')}")

# 4. OneHotEncoder 저장
joblib.dump(onehot_encoder, os.path.join(output_dir, 'onehot_encoder.pkl'))
print(f"✅ OneHotEncoder 저장 완료: {os.path.join(output_dir, 'onehot_encoder.pkl')}")

# 5. 수치 특징 컬럼명 저장
with open(os.path.join(output_dir, 'numerical_cols.pkl'), 'wb') as f:
    pickle.dump(numerical_cols, f)
print(f"✅ 수치 특징 컬럼명 저장 완료: {os.path.join(output_dir, 'numerical_cols.pkl')}")